# Signify CNN Model

In [2]:
import sys
sys.path.append('../')

## Importing the libraries

In [3]:
import cv2
import joblib
import kaggle
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from utils.apply_filter import apply_filter
from utils.test_train_split import test_train_split

## Downloading Dataset

In [8]:
# Get the current working directory
cwd = os.getcwd()

# Get the folder path relative to the current working directory
destination_folder = os.path.join(cwd, '..', 'dataset')

# Setting the API credentials and using the API command to download the dataset
kaggle.api.authenticate()
kaggle.api.dataset_download_files('prathumarikeri/indian-sign-language-isl', path=destination_folder, unzip=True)

## Applying Filter on the dataset images

In [9]:
# Folder to apply the filter on
data_dir = os.path.join(cwd, '..', 'dataset/Indian')

# Iterate over the subdirectories
for subdir_name in os.listdir(data_dir):
    subdir_path = os.path.join(data_dir, subdir_name)
    if os.path.isdir(subdir_path):
        # Iterate over the files in the subdirectory
        for file_name in os.listdir(subdir_path):
            if file_name.endswith(".jpg"):  # Filter only for JPG files
                file_path = os.path.join(subdir_path, file_name)
                apply_filter(file_path)

## Spliting dataset into training and testing set

In [10]:
cwd = os.getcwd()
data_dir = os.path.join(cwd, '..', 'dataset/Indian')
test_train_split(data_dir)

Copying files: 42745 files [04:39, 153.01 files/s]


## Importing and preprocessing data

### Preprocessing the Training Set

In [6]:
cwd = os.getcwd()
data_dir = os.path.join(cwd, '..', 'dataset/Indian')
train_dir = os.path.join(data_dir, 'train')

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

training_set = train_datagen.flow_from_directory(train_dir,
                                                 target_size = (128, 128),
                                                 batch_size = 32,
                                                 color_mode='grayscale',
                                                 class_mode = 'categorical')

Found 8640 images belonging to 9 classes.


### Preprocessing the Testing Set

In [7]:
test_dir = os.path.join(data_dir, 'val')

test_datagen = ImageDataGenerator(rescale = 1./255)

test_set = test_datagen.flow_from_directory(test_dir,
                                            target_size = (128,128),
                                            batch_size = 32,
                                            color_mode='grayscale',
                                            class_mode = 'categorical')

Found 2160 images belonging to 9 classes.


## Building the CNN model

### Initializing the CNN object

In [9]:
cnn = tf.keras.models.Sequential()

### Adding first convolution layer

In [10]:
cnn.add(tf.keras.layers.Conv2D(filters=32,
                               kernel_size=3,
                               activation="relu",
                               input_shape=[128, 128, 1]))

### Adding first max pooling layer

In [11]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding second convolution layer

In [12]:
cnn.add(tf.keras.layers.Conv2D(filters=32,
                               kernel_size=5,
                               activation="relu"))

### Adding second max pooling layer

In [13]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Flattening the matrix of features

In [14]:
cnn.add(tf.keras.layers.Flatten())

### Adding the fully connected layer

In [15]:
cnn.add(tf.keras.layers.Dense(units=128, activation="relu"))
cnn.add(tf.keras.layers.Dropout(0.40))
cnn.add(tf.keras.layers.Dense(units=96, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.40))
cnn.add(tf.keras.layers.Dense(units=64, activation='relu'))

### Adding the output layer

In [20]:
cnn.add(tf.keras.layers.Dense(units=9, activation="softmax"))

## Training the model

In [21]:
cnn.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [22]:
cnn.fit(training_set, validation_data = test_set, epochs=3)

Epoch 1/3
270/270 [==============================] - 187s 687ms/step - loss: 2.1993 - accuracy: 0.1096 - val_loss: 2.1976 - val_accuracy: 0.1111
Epoch 2/3
270/270 [==============================] - 131s 486ms/step - loss: 2.1978 - accuracy: 0.1054 - val_loss: 2.1973 - val_accuracy: 0.1111
Epoch 3/3
270/270 [==============================] - 132s 490ms/step - loss: 2.1976 - accuracy: 0.1036 - val_loss: 2.1972 - val_accuracy: 0.1111
